In [1]:
import pytorch_lightning as pl
import torch
import torch.nn as nn

from models import *

from torchmetrics import Accuracy

from pytorch_lightning.callbacks.progress import TQDMProgressBar, RichProgressBar


In [2]:
class Classifier(pl.LightningModule):
    def __init__(self, model, optimizer):
        super().__init__()

        self.model = model
        self.optimizer = optimizer

        self.train_acc = Accuracy(task="multiclass", num_classes=10)
        self.valid_acc = Accuracy(task="multiclass", num_classes=10)
        self.test_acc = Accuracy(task="multiclass", num_classes=10)

    def forward(self, x):
        x = self.model(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(self(x), y)
        preds = torch.argmax(logits, dim=1)
        self.train_acc.update(preds, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss
    
    # def on_train_epoch_end(self, outs):
    #     self.log("train_acc", self.train_acc.compute())

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(self(x), y)
        preds = torch.argmax(logits, dim=1)
        self.valid_acc.update(preds, y)
        self.log("valid_loss", loss, prog_bar=True)
        self.log("valid_acc", self.valid_acc.compute(), prog_bar=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
    
        loss = nn.functional.cross_entropy(self(x), y)
        preds = torch.argmax(logits, dim=1)

        self.test_acc.update(preds, y)
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", self.test_acc.compute(), prog_bar=True)
        return loss
    
    def configure_optimizers(self):
        return self.optimizer


In [3]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms


class MnistDataModule(pl.LightningDataModule):
    def __init__(self, data_path='./', batch_size = 128, num_workers = 8):
        super().__init__()
        self.data_path = data_path
        self.transform = transforms.Compose([transforms.ToTensor()])

        self.batch_size = batch_size
        self.num_workers = num_workers

    def prepare_data(self):
        MNIST(root=self.data_path, download=True)

    def setup(self, stage=None):
        mnist_all = MNIST(
            root=self.data_path,
            train=True,
            transform=self.transform,
            download=False
        )

        self.train, self.val = random_split(
            mnist_all, [50000, 10000], generator=torch.Generator().manual_seed(1)
        )

        self.test = MNIST(
            root=self.data_path,
            train=False,
            transform=self.transform,
            download=False
        )

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size, num_workers=self.num_workers)


In [4]:
torch.manual_seed(1)
mnist_dm = MnistDataModule(data_path="./data/")

In [5]:
model = FlattenModel(nn.Linear(28*28, 512),
                     nn.ReLU(),
                     nn.Linear(512, 256),
                     nn.ReLU(),
                     nn.Linear(256, 128),
                     nn.ReLU(),
                     nn.Linear(128, 10),
                     )

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

mnistclassifier = Classifier(model=model, optimizer=optimizer)

trainer = pl.Trainer(max_epochs=40, callbacks=[RichProgressBar(refresh_rate=20)])

trainer.fit(model=mnistclassifier, datamodule=mnist_dm)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name      ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model     │ FlattenModel       │  567 K │
│ 1 │ train_acc │ MulticlassAccuracy │      0 │
│ 2 │ valid_acc │ MulticlassAccuracy │      0 │
│ 3 │ test_acc  │ MulticlassAccuracy │      0 │
└───┴───────────┴────────────────────┴────────┘

Trainable params: 567 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 567 K                                                                                                
Total estimated model params size (MB): 2

2023-03-27 13:59:02.847035: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Output()

`Trainer.fit` stopped: `max_epochs=40` reached.


In [6]:
cnn_layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
        )

linear_layers = nn.Sequential(
            nn.Linear(7 * 7 * 32, 128),
            nn.ReLU(),
            nn.Dropout(p = 0.2),
            nn.Linear(128, 10),
        )

model = ConvModel(cnn_layers, linear_layers)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

mnistclassifier = Classifier(model=model, optimizer=optimizer)

trainer = pl.Trainer(max_epochs=40, callbacks=[RichProgressBar(refresh_rate=20)])

trainer.fit(model=mnistclassifier, datamodule=mnist_dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name      ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model     │ ConvModel          │  202 K │
│ 1 │ train_acc │ MulticlassAccuracy │      0 │
│ 2 │ valid_acc │ MulticlassAccuracy │      0 │
│ 3 │ test_acc  │ MulticlassAccuracy │      0 │
└───┴───────────┴────────────────────┴────────┘

Trainable params: 202 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 202 K                                                                                                
Total estimated model params size (MB): 0

Output()

`Trainer.fit` stopped: `max_epochs=40` reached.


In [7]:
trainer.test(model=mnistclassifier, datamodule=mnist_dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9852431416511536     │
│         test_loss         │    0.04523905739188194    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.04523905739188194, 'test_acc': 0.9852431416511536}]